# Image Classification: Fashion-MNIST

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Load the data

In [2]:
# Load the data

# Get current working directory
current_dir = os.getcwd() 

# Append data/mnist.npz to the previous path to get the full path
data_path = os.path.join(current_dir, "data/mnist.npz") 

# Get only training set
(training_images, training_labels), _ = tf.keras.datasets.mnist.load_data(path=data_path) 

## Pre-processing the data

In [24]:
def reshape_and_normalize(images):
    
    # Reshape the images to add an extra dimension
    images = images.reshape(images.shape + (1,))
    
    # Normalize pixel values
    images = images / 255
    
    return images

In [25]:
# Reload the images
(training_images, _), _ = tf.keras.datasets.mnist.load_data(path=data_path) 

# Apply function
training_images = reshape_and_normalize(training_images)

print(f"Maximum pixel value after normalization: {np.max(training_images)}\n")
print(f"Shape of training set after reshaping: {training_images.shape}\n")
print(f"Shape of one image after reshaping: {training_images[0].shape}")


Maximum pixel value after normalization: 1.0

Shape of training set after reshaping: (60000, 28, 28, 1)

Shape of one image after reshaping: (28, 28, 1)


## Defining callback

In [35]:
class myCallback(keras.callbacks.Callback):
    # Define the method that checks the accuracy at the end of each epoch
    def on_epoch_end(self, epoch, logs=None):
        # Check accuracy
        if(logs.get('accuracy') >= 0.995):
            # Stop if threshold is met
            print("\nReached 99.5% accuracy so cancelling training!")
            self.model.stop_training = True

## Convolutional Model

In [36]:
def convolutional_model():
    # Define the model
    model = tf.keras.models.Sequential([keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
                                        keras.layers.MaxPooling2D(2,2),
                                        keras.layers.Conv2D(64, (3,3), activation='relu'),
                                        keras.layers.MaxPooling2D(2,2),
                                        keras.layers.Flatten(),
                                        keras.layers.Dense(128, activation='relu'),
                                        keras.layers.Dense(10, activation='softmax')
                                       ])

    # Compile the model
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy']) 
        
    return model

In [37]:
# Save untrained model
model = convolutional_model()

# Instantiate the callback class
callbacks = myCallback()

# Train model
history = model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 87s 46ms/step - loss: 0.1234 - accuracy: 0.9627
Epoch 2/10
1875/1875 [==============================] - 86s 46ms/step - loss: 0.0406 - accuracy: 0.9876
Epoch 3/10
1875/1875 [==============================] - 87s 46ms/step - loss: 0.0277 - accuracy: 0.9910
Epoch 4/10
1875/1875 [==============================] - 87s 46ms/step - loss: 0.0203 - accuracy: 0.9935
Epoch 5/10
1875/1875 [==============================] - ETA: 0s - loss: 0.0142 - accuracy: 0.9953
Reached 99.5% accuracy so cancelling training!
1875/1875 [==============================] - 86s 46ms/step - loss: 0.0142 - accuracy: 0.9953


In [38]:
print(f"Your model was trained for {len(history.epoch)} epochs")

Your model was trained for 5 epochs
